## Advanced RAG System Using Llama2

### Установка библиотек

Pypdf для работы с pdf-документами.
Идея заключается в том что, если у нас много PDF-документов, мы должны иметь возможность загружать эти PDF-документы, индексировать эти PDFдокументы и выполнять запросы вместе с моделью Lama2.

In [ ]:
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 1.8 MB/s eta 0:00:00


Transformers предоставляет API и инструменты для простой загрузки и настройки современных предварительно обученных моделей.

Einops - Эта библиотека предоставляет инструменты для работы с тензорами, в частности, для изменения формы, перестановки и уменьшения размеров в многомерных массивах (тензорах).

Accelerate - это библиотека, которая позволяет запускать один и тот же основной код на распределенной конфигурации. Нужна чтобы ускорить процессы.

LangChain - это фреймворк для разработки приложений на базе LLM.

Bitsandbytes предоставляет функции, позволяющие значительно сократить потребление памяти для ответов и обучения. большинство моделей большинство моделей llm содержат 16 бит, но поскольку мы в Google collab, то будем квантовать их до 4х бит.

In [ ]:
!pip install -q transformers einops accelerate langchain bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 977.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.3/297.3 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.8/276.8 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

Sentence_transformers для работы с эмбеддингами.

In [ ]:
!pip install install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 2.0 MB/s eta 0:00:00


LlamaIndex (GPT Index) - фреймворк, который предоставляет интерфейс для подключения LLM к внешним данным.

In [ ]:
!pip install llama_index

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.0/136.0 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 9.6 MB/s eta 0:00:00


Сопутсвующая к llamaindex библиотека для работы с LLM huggingface.

In [ ]:
!pip install llama-index-llms-huggingface

Импорт компонентов из библиотек:

* SimpleDirectoryReader для загрузки локальных файлов.
* VectorStoreIndex для индексирования документов и создания векторов.
* ServiceContext для соединения LLM и Эмбединг-модели.
* HuggingfaceLLM для взаимодействия с моделями Hugging Face.
* SimpleInputPrompt для создания prompt format (формата промпта).

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.prompts.prompts import SimpleInputPrompt

### Настройка RAG и LLM

Подключение гугл диска для хранения файлов, чтобы они не удалялись при отключении от среды выполнения.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Загрузка локальных файлов с помощью SimpleDirectoryReader.

In [ ]:
documents=SimpleDirectoryReader("/content/drive/MyDrive/data").load_data()
documents

[Document(id_='3dd8f823-4941-48ce-ae78-dc378feb2a52', embedding=None, metadata={'page_label': '1', 'file_name': '00_ViPNet SIES DevKit. Развертывание.pdf', 'file_path': '/content/drive/MyDrive/data/00_ViPNet SIES DevKit. Развертывание.pdf', 'file_type': 'application/pdf', 'file_size': 1407347, 'creation_date': '2024-04-06', 'last_modified_date': '2024-04-06'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='Комплект разработчика ViPNet SIES . Развёртывание   |  1  \nКомплект разработчика \nViPNet SIES  \nРазвёртывание  \n \n \n ', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'),
 Document(id_='21d24909-8d4d-48df-ae6c-026fb4ca9030', embedding=None, metadata={

Проводим очистку данных. Удаляем документы, которые не несут какой-либо информации и мешают RAG при поиске данных.

In [ ]:
print("Количество документов до очистки данных = ", len(documents))
for doc in documents:

    if "................................" in doc.text:
        documents.remove(doc)

    elif "©" in doc.text:
        documents.remove(doc)

    elif "О документе" in doc.text:
        documents.remove(doc)

    elif "Обратная связь" in doc.text:
        documents.remove(doc)

    elif "Содержимое документа носит исключительно информационный характер" in doc.text:
        documents.remove(doc)


print("Количество документов после очистки данных = ", len(documents))

Количество документов до очистки данных =  3179
Количество документов после очистки данных =  3050


Создание propmt template (шаблона проспта) и prompt format (формата промпта) (используем формат промпта, поддерживаемый LLama 2) (при использовании промпта для LLama2, этот промпт необходимо давать в формате).
* USER - плейсхолдер роли человека, взаимодействующего с моделью, в данном случае пользователя, который вводит запрос.
* {query_str} - плейсхолдер запроса пользователя.
* ASSISTANT - плейсхолдер роли модели, которая отвечает на запросы пользователя.

**В этот раз системный промпт другой.**

In [ ]:
system_prompt="""
You work in customer support at ViPNet SIES, providing helpful information to our valued customers. Your job requires you to be friendly, polite, and professional at all times while answering customer questions. It's important that you provide only factual information, as this helps build trust and ensures that customers receive accurate answers.
To better understand the context of the question, please refer to the information provided below. Based on this information and without any prior knowledge of the specific situation, kindly provide an answer to the following question:
"""
## Default format supportable by LLama2
query_wrapper_prompt=SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

Входим в аккаунт huggingface с помощью токена.

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' creden

Загрузка, настройка и квантование модели LLM.



*    context_window - количество токенов, которые модель может использовать в качестве входных данных при генерации ответов.
*    max_new_tokens - максимальное количество новых токенов, которые должна генерировать модель.
*    generate_kwargs - дополнительные аргументы для генерации.
** temperature - параметр определяет случайность генерации токенов (насколько модель креативна). Более низкие температуры приводят к меньшей случайности, обеспечивая более детерминированные выходные данные.
**   do_sample - указывает, следует ли использовать выборку при генерации токенов. Если установлено значение True, модель будет произвольно выбирать токены из распределения вероятности по всему словарю с различными корректировками, зависящими от конкретной стратегии. В противном случае используется жадное декодирование.
Значение true привносит элемент случайности, привнося творческий подход в выходные данные LLM.
*    system_prompt - системный промпт (шаблон промпта). Системный промпт определяет, как модельинтерпретируют запросы пользователей и отвечает на них. Эти тщательно разработанные инструкции служат ориентиром для модели, направляя её поведение и гарантируя соответствие генерируемых результатов намеченным целям.
*    query_wrapper_prompt - формат промпта (обёртка вокруг инпута). Формат промпта служит основой для представления запроса пользователя модели.
*    tokenizer_name - имя/путь токенизатора, который будет использоваться.
*    model_name - имя/путь предварительно обученной языковой модели, которая будет использоваться.
*    device_map - Устройство, на котором будет загружаться и запускаться модель. Auto означает, что модель автоматически выберет доступное аппаратное обеспечение (CPU или GPU) для запуска и вычислений.
*    model_kwargs - дополнительные аргументы для инициализации модели.
** torch_dtype - тип данных, используемый для вычислений в рамках модели.
**  load_in_8bit - указывает, следует ли загружать параметры модели с 8-битной точностью.

In [ ]:
import torch

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="NousResearch/Llama-2-7b-chat-hf",
    model_name="NousResearch/Llama-2-7b-chat-hf",
    device_map="auto",
    model_kwargs={"torch_dtype": torch.float16 , "load_in_8bit":True}
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

Настройка модели для эмбеддинга.

HuggingFaceEmbeddings нужна для получения sentence-transformers с HuggingFace.

LangchainEmbedding преобразует слова в числовые массивы или векторы с помощью заданной модели. Эти векторы отображают взаимосвязи между словами. Набор чисел в этих векторах служит многомерной основой для измерения сходства.

all-mpnet-base-v2 - модель преобразования предложений: она преобразовывает предложения и абзацы в плотном векторном пространстве с 768 измерениями.

In [ ]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.core import ServiceContext
from llama_index.legacy.embeddings.langchain import LangchainEmbedding

embed_model=LangchainEmbedding(
    HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))

[nltk_data] Downloading package stopwords to
[nltk_data]     /usr/local/lib/python3.10/dist-
[nltk_data]     packages/llama_index/legacy/_static/nltk_cache...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     /usr/local/lib/python3.10/dist-
[nltk_data]     packages/llama_index/legacy/_static/nltk_cache...
[nltk_data]   Unzipping tokenizers/punkt.zip.


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Связываем эмбединг-модель и LLM-модель.

ServiceContext - это набор часто используемых ресурсов, используемых на этапе индексации и запроса в пайплайне/приложении LlamaIndex. Его можно использовать для настройки глобальной конфигурации, а также локальных конфигураций на определенных участках пайплайна.

from_defaults - значит, все не заданные параметры остаются со значениями по умолчанию:
* chunk_size - максимальное количество символов, которое содержит чанк.
* chunk_overlap - количество символов, которые должны перекрываться между двумя соседними чанками. chunk_overlap позволяет избежать резкое прерывание на середине ключевой идеи или фрагмента информации, которые мы хотим передать модели.
* llm - LLM-модель.
* embed_model - эмбединг-модель.

**В этот раз мы уменьшили chunk_size и добавили chunk_overlap.**

In [ ]:
service_context=ServiceContext.from_defaults(
    chunk_size=256,
    chunk_overlap=25,
    llm=llm,
    embed_model=embed_model
)

<ipython-input-32-35e771eb0a43>:1: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context=ServiceContext.from_defaults(


In [ ]:
service_context

ServiceContext(llm_predictor=LLMPredictor(system_prompt=None, query_wrapper_prompt=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>), prompt_helper=PromptHelper(context_window=4096, num_output=256, chunk_overlap_ratio=0.1, chunk_size_limit=None, separator=' '), embed_model=LangchainEmbedding(model_name='sentence-transformers/all-mpnet-base-v2', embed_batch_size=10, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7f603c5d3730>), transformations=[SentenceSplitter(include_metadata=True, include_prev_next_rel=True, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7f603c5d3730>, id_func=<function default_id_func at 0x7f611a97ff40>, chunk_size=256, chunk_overlap=25, separator=' ', paragraph_separator='\n\n\n', secondary_chunking_regex='[^,.;。？！]+[,.;。？！]?')], llama_logger=<llama_index.core.service_context_elements.llama_logger.LlamaLogger object at 0x7f6048d35d20>, callback_manager=<llama_index.core.callbacks.base

Конвертация документов в индексы с использованием эмбеддинг-модели и их сохранение в векторном виде вместе с нашей LLM. service_context предоставляет эмбединг-модель и LLM-модель, и настройки индексирования.

In [ ]:
index=VectorStoreIndex.from_documents(documents,service_context=service_context)

In [ ]:
index

Преобразование векторного хранилища в query_engine (движок для обработки запросов).

Query engine - это универсальный интерфейс, который позволяет задавать вопросы по нашим данным. Query engine принимает запрос на естественном языке и возвращает развернутый ответ. Чаще всего (но не всегда) он основан на одном или нескольких индексах с помощью средств извлечения.

In [ ]:
query_engine=index.as_query_engine()

### Тестирование LLM с оптимизированным RAG

Задание вопросов Query engine (модели LLM с RAG).

In [ ]:
response=query_engine.query("What is SIES?")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [ ]:
print(response)

SIES is an acronym for "Secure Internet of Things (IoT) Embedded System". It refers to a secure and reliable embedded system designed for IoT applications, providing a comprehensive solution for secure data transmission and storage. The SIES Core SDK 2.0 is a software development kit that enables developers to build secure IoT applications using the SIES embedded system.


In [ ]:
response=query_engine.query("what is SIES Workstation used for?")

In [ ]:
print(response)

SIES Workstation is used for local management and monitoring of ViPNet SIES Core and ViPNet SIES Unit. It allows for the delivery of commands and monitoring of the system without using industrial communication channels.
